In [31]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [32]:
from LagsCreator import LagsCreator

In [33]:
data = np.array([["a1", "a2", "a3", "a4", "a5", "a6", "a7", "a8", "a9", "a10", "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20"], 
                 ["b1", "b2", "b3", "b4", "b5", "b6", "b7", "b8", "b9", "b10", "b11", "b12", "b13", "b14", "b15", "b16", "b17", "b18", "b19", "b20"], 
                 ["c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9", "c10", "c11", "c12", "c13", "c14", "c15", "c16", "c17", "c18", "c19", "c20"]])

In [34]:
#data = np.random.rand(data.shape[0], data.shape[1])

In [35]:
df = pd.DataFrame(np.transpose(data), index = pd.date_range("2018-08-22", periods = 20), columns = pd.MultiIndex.from_product([["Group 1"], ["A", "B", "C"]], names = ["Group", "Time-series"]))

In [36]:
df

Group       Group 1          
Time-series       A    B    C
2018-08-22       a1   b1   c1
2018-08-23       a2   b2   c2
2018-08-24       a3   b3   c3
2018-08-25       a4   b4   c4
2018-08-26       a5   b5   c5
2018-08-27       a6   b6   c6
2018-08-28       a7   b7   c7
2018-08-29       a8   b8   c8
2018-08-30       a9   b9   c9
2018-08-31      a10  b10  c10
2018-09-01      a11  b11  c11
2018-09-02      a12  b12  c12
2018-09-03      a13  b13  c13
2018-09-04      a14  b14  c14
2018-09-05      a15  b15  c15
2018-09-06      a16  b16  c16
2018-09-07      a17  b17  c17
2018-09-08      a18  b18  c18
2018-09-09      a19  b19  c19
2018-09-10      a20  b20  c20

In [37]:
lags_dict = dict()
# Define lags for each indicator.
lags_dict["A"] = 3
lags_dict["B"] = 4
lags_dict["C"] = 0

In [38]:
import time

In [39]:
a=time.time()
creator = LagsCreator(df, lags_dict, "A")
X_train, y_train, X_val, y_val, X_test = creator.to_supervised(4, single_step=True,return_dataframe = True,h=2, 
                                                               validation=True, feature_time=True, return_single_level = True, 
                                                               dtype = object)
b=time.time()
print(b-a)

0.06286883354187012


In [169]:
train, val, test = creator.visualization(boundaries = False)

AttributeError: 'Timestamp' object has no attribute 'format'

In [161]:
train[0]

ValueError: Result of <function LagsCreator.highlight_cells.<locals>.<lambda> at 0x000001D44A725158> must have identical index and columns as the input

In [174]:
df.index.

SyntaxError: invalid syntax (<ipython-input-174-c105f4bdc56d>, line 1)

In [121]:
X_train

,A | x(t-2),A | x(t-1),A | x(t),B | x(t-3),B | x(t-2),B | x(t-1),B | x(t),C | x,Day | x,Month | x,Year | x
0,a2,a3,a4,b1,b2,b3,b4,c4,26,8,2018
1,a3,a4,a5,b2,b3,b4,b5,c5,27,8,2018
2,a4,a5,a6,b3,b4,b5,b6,c6,28,8,2018
3,a5,a6,a7,b4,b5,b6,b7,c7,29,8,2018
4,a6,a7,a8,b5,b6,b7,b8,c8,30,8,2018
5,a7,a8,a9,b6,b7,b8,b9,c9,31,8,2018
6,a8,a9,a10,b7,b8,b9,b10,c10,1,9,2018
7,a9,a10,a11,b8,b9,b10,b11,c11,2,9,2018
8,a10,a11,a12,b9,b10,b11,b12,c12,3,9,2018
9,a11,a12,a13,b10,b11,b12,b13,c13,4,9,2018


In [122]:
y_train

,A | x(t+1)
0,a5
1,a6
2,a7
3,a8
4,a9
5,a10
6,a11
7,a12
8,a13
9,a14


In [214]:
X_test

,A | x(t-2),A | x(t-1),A | x(t),B | x(t-3),B | x(t-2),B | x(t-1),B | x(t),C | x,Day | x,Month | x,Year | x
Group,,,,,,,,,,,
Group 1,a18,a19,a20,b17,b18,b19,b20,c20,10,9,2018


AttributeError: 'Index' object has no attribute 'strftime'

In [216]:
len(train)

16

In [217]:
train[5]

Time-series,A,B,C
2018-08-22 00:00:00,a1,b1,c1
2018-08-23 00:00:00,a2,b2,c2
2018-08-24 00:00:00,a3,b3,c3
2018-08-25 00:00:00,a4,b4,c4
2018-08-26 00:00:00,a5,b5,c5
2018-08-27 00:00:00,a6,b6,c6
2018-08-28 00:00:00,a7,b7,c7
2018-08-29 00:00:00,a8,b8,c8
2018-08-30 00:00:00,a9,b9,c9
2018-08-31 00:00:00,a10,b10,c10


In [117]:
val[-1]

TypeError: 'NoneType' object is not subscriptable

In [47]:
test[0]

Time-series,A,B,C
2018-08-22 00:00:00,a1,b1,c1
2018-08-23 00:00:00,a2,b2,c2
2018-08-24 00:00:00,a3,b3,c3
2018-08-25 00:00:00,a4,b4,c4
2018-08-26 00:00:00,a5,b5,c5
2018-08-27 00:00:00,a6,b6,c6
2018-08-28 00:00:00,a7,b7,c7
2018-08-29 00:00:00,a8,b8,c8
2018-08-30 00:00:00,a9,b9,c9
2018-08-31 00:00:00,a10,b10,c10
